In [ ]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
sns.set(font_scale=1.3)

### Линейная регрессия: практика
Перед началом работы догрузим необходимые нам функции



In [ ]:
from sklearn.datasets import load_diabetes
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

Рассмотрим данные исследования 2004 года о зависимости риска развития диабета от пола, возраста, индекса массы тела, среднего кровяного давления и других показателей.



#### Загрузка данных

In [ ]:
data = load_diabetes()
df = pd.DataFrame(data.data, columns = data.feature_names)
df.head()

Функция `sklearn.datasets.load_diabetes()` возвращает словарь. В поле `data` записана матрица регрессоров, в которой данные предварительно центрированы и нормированы. В поле `target` записана мера прогрессирования заболевания в течении года. В поле `DESCR` можно прочитать подробнее о данных.

Посмотрим на описание датасета.

In [ ]:
print(data['DESCR'])

Поле `data` содержит матрицу размера 442 $\times$ 10, где 442 — количество пациентов, а 10 — количество признаков (возраст, пол, и т.д.).
Строки матрицы соответствуют пациентам, столбцы — признакам.

In [ ]:
data['data'].shape

Целевая переменная $-$ мера прогрессирования заболевания в течении года.

In [ ]:
data['target'].shape

Создайте матрицу регрессоров $X$ (data) и столбец наблюдений $y$ (целевая переменная).

In [ ]:
X, y = df.values, data.target

print(X.shape, y.shape)

#### Обучение моделей

Разбейте данные случайно на две части — обучающую и тестовую в соотношении 80:20.

Если что-то забыли или что-то не понятно, можно почитать справку: https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html

При разбиении датасета стоит зафиксировать случайность для воспроизводимости результатов, поставив `random_state=42` в функцию разбиения (можете ставить любое число, необязательно именно 42)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

Заведите модель линейной регрессии из `sklearn` и обучите ее по обучающей части данных.

In [ ]:
model = LinearRegression(fit_intercept = True)  # объявляем модель
model.fit(X_train, y_train)  # обучаем

Посмотрите на результат обучения. Выведите коэффициенты перед признаками и свободный коэффициент.

In [ ]:
print("Веса: ", model.coef_)
print("Свободный член: ", model.intercept_)

Выполните предсказание модели на тестовой выборке.

In [ ]:
y_pred = model.predict(X_test)

Реализуйте метрики MSE, MAE, MAPE без использования `sklearn` и других готовых реализаций.

Пусть $Y_1, ..., Y_n$ &mdash; истинные значения, а $\widehat{Y}_1, ..., \widehat{Y}_n$ &mdash; предсказания.


Метрика MSE (mean squared error) определяется как
$$MSE = \frac{1}{n}\sum_{i=1}^n \left(Y_i - \widehat{Y}_i\right)^2.$$

Метрика **MAE** (*mean absolute error*), определяемая как
$$MAE = \frac{1}{n}\sum_{i=1}^n \left|Y_i - \widehat{Y}_i\right|.$$

Метрика **MAPE** (*mean absolute percentage error*), определяемая как
$$MAPE = 100\% \cdot \frac{1}{n}\sum_{i=1}^n \frac{\left|Y_i - \widehat{Y}_i\right|}{Y_i}.$$


In [ ]:
def mean_squared_error(y_true, y_pred):
    return sum([(y_pred[i] - y_true[i])**2 for i in range(len(y_pred))])/len(y_pred)

def mean_absolute_error(y_true, y_pred):
    return sum([abs(y_pred[i] - y_true[i]) for i in range(len(y_pred))])/len(y_pred)

def mean_absolute_percentage_error(y_true, y_pred):
    return sum([abs(y_pred[i] - y_true[i])/y_true[i] for i in range(len(y_pred))])/len(y_pred) * 100

Посчитайте MSE, MAE, MAPE на тестовой выборке и выведите с точностью до трех знаков после запятой.

In [ ]:
MSE = mean_squared_error(y_test, y_pred)
MAE = mean_absolute_error(y_test, y_pred)
MAPE = mean_absolute_percentage_error(y_test, y_pred)

print(f'MSE: {MSE:.3f}')
print(f'MAE: {MAE:.3f}')
print(f'MAPE: {MAPE:.3f}')

### Различные разбиения

Разбейте данные на тренировочную и тестовую выборки еще раз: измените пераметр `random_state`. Для каждого из разбиений обучите модель и получите метрики качества. Меняются ли эти метрики, если менять разбиение данных?

In [ ]:
mse = []
mae = []
mape = []
for state in [1, 495, 324, 8, 0, 900]:

    # разбиение данных
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = state)

    # обучение модели
    model = LinearRegression(fit_intercept = True)
    model.fit(X_train, y_train)

    # предсказание модели
    y_pred = model.predict(X_test)

    # подсчет метрики на тестовой выборке
    mse.append(mean_squared_error(y_test, y_pred))
    mae.append(mean_absolute_error(y_test, y_pred))
    mape.append(mean_absolute_percentage_error(y_test, y_pred))

In [ ]:
mse, mae, mape

**Вывод:** выбор случайного разбиения данных влияет на оценки производительности модели.